<h1>Synthetic Lethality Interaction Network Bias</h1>

<p>The purpose of this notebook is to demonstrate the influence of node degree on the performance of graph machine learning. We use the same graph that was presented in the notebook
    <a href="https://github.com/monarch-initiative/negativeExampleSelection/blob/main/SliNetwork.ipynb" target="__blank">SliNetwork</a>.</p>

In [1]:
from grape.datasets.kghub import SLDB
from grape.datasets.string import HomoSapiens
import pandas as pd
from tqdm.auto import tqdm
from grape.embedders import embed_graph, DeepWalkSkipGramEnsmallen, Node2VecSkipGramEnsmallen, Node2VecCBOWEnsmallen, Node2VecGloVeEnsmallen
from grape import GraphVisualizer
from grape.edge_prediction import (
    DecisionTreeEdgePrediction,
    edge_prediction_evaluation,
    PerceptronEdgePrediction,
    KipfGCNEdgePrediction,
    RandomForestEdgePrediction,
)
import pandas as pd
from collections import Counter
from operator import itemgetter
from tqdm.auto import tqdm, trange
from typing import List, Generator, Optional
import matplotlib.pyplot as plt
from matplotlib.axes import Axes
from matplotlib.figure import Figure
from IPython.display import HTML
import warnings
warnings.filterwarnings('ignore')

In [2]:
%%time
sldb = SLDB()
stringGraph = HomoSapiens()
path = "graphs/string/HomoSapiens/links.v11.5/9606.protein.info.v11.5.txt"
data = pd.read_csv(path, sep="\t")
remapping_string = dict(zip(data[data.columns[0]], data[data.columns[1]]))
string_graph = stringGraph \
    .remove_node_types() \
    .filter_from_names(min_edge_weight=700) \
    .remove_edge_weights() \
    .remap_from_node_names_map(remapping_string) \
    .set_all_node_types("Gene") \
    .set_all_edge_types("PPI") \
    .remove_disconnected_nodes()
path = "graphs/kghub/SLDB/20220522/sldb/merged-kg_nodes.tsv"
data = pd.read_csv(path, sep="\t")
remapping_sli = dict(zip(data[data.columns[0]], data[data.columns[2]]))

# We load the SLI graph
sli_graph = SLDB() \
    .remap_from_node_names_map(remapping_sli) \
    .set_all_node_types("Gene") \
    .set_all_edge_types("SLI")
composite_graph = (sli_graph | string_graph) \
    .remove_components(top_k_components=1)

composite_graph.enable()

sli_subgraph = composite_graph.filter_from_names(
    edge_type_names_to_keep=["SLI"]
)
sli_subgraph.enable()

CPU times: user 24.1 s, sys: 2.34 s, total: 26.4 s
Wall time: 11.6 s


<h2>Differential degree distribution of genes with  SLIs</h2>
<p>As could be seen in the SliNetwork notebook, the SLI subnetwork displays a scale-free distribution. We show here that the nodes of the SLI network differ in the mean degree compared to other nodes in the network.</p>

In [3]:
from statistics import mean
node_labels = composite_graph.get_node_names()
node_degrees = composite_graph.get_node_degrees()
node_degree_d = dict(zip(node_labels, node_degrees))
sli_label_set = set(sli_graph.get_node_names())
mean_sli_degree = mean([degree for gene, degree in node_degree_d.items() if gene in sli_label_set])
mean_nonsli_degree = mean([degree for gene, degree in node_degree_d.items() if gene not in sli_label_set])
print(f"Mean degree of genes in SLI set: {mean_sli_degree} and genes not in SLI set {mean_nonsli_degree}")

Mean degree of genes in SLI set: 71 and genes not in SLI set 25


In [4]:
results_list = []

<p>In the following, we show that classifiers can exploit this difference to make predictions based solely on node degree.</p>

In [5]:
def get_dict_with_values(df, name, normalized):
    """
    convenience function to group results in a dictionary
    """
    results = df.groupby(['evaluation_mode',]).agg(['mean','std'])[['balanced_accuracy']]
    test_results = results.loc['test'].values
    train_results = results.loc['train'].values
    d = {'name': name, 'test.mean' : test_results[0], 'test.std': test_results[1], 
       'train.mean' : train_results[0], 'train.std': train_results[1]}
    if normalized:
        d['normalized'] = "True"
    else:
        d['normalized'] = "False"
    return d

<h2>Prediction on the basis of node degree</h2>
<p>Here we show that a simple perceptron is able to achieve a reasonable prediction accuracy in a  cross-validation setting in which we do not sample edges in training or validation based on node degree. This analysis is intended to demonstrate the potential for overfitting based on node degree.</p>

In [6]:
results_degree = edge_prediction_evaluation(
    holdouts_kwargs=dict(train_size=0.80),
    graphs=sldb,
    number_of_holdouts=10,
    models=PerceptronEdgePrediction(
        edge_features=["Degree"],
        use_scale_free_distribution=False,
        number_of_edges_per_mini_batch=16
    ),
    use_scale_free_distribution=False,
)

Evaluating on SLDB:   0%|                                | 0/10 [00:00<?, ?it/s]

In [7]:
results_degree.groupby([
    "evaluation_mode",
    "use_scale_free_distribution"
]).agg(["mean", "std"])[["balanced_accuracy"]]

balanced_accuracy          
                                                         mean       std
evaluation_mode use_scale_free_distribution                            
test            False                                0.770041  0.034245
train           False                                0.907720  0.005235

<h2>Prediction on the basis of node degree (corrrected)</h2>
<p>Here, we perform the same analysis as above, but sample edges in training or validation based on node degree.</p>

In [8]:
unbiased_results_degree = edge_prediction_evaluation(
    holdouts_kwargs=dict(train_size=0.80),
    graphs=sldb,
    number_of_holdouts=10,
    models=PerceptronEdgePrediction(
        edge_features=["Degree"],
        use_scale_free_distribution=True,
        number_of_edges_per_mini_batch=16
    ),
    use_scale_free_distribution=True,
)

Evaluating on SLDB:   0%|                                | 0/10 [00:00<?, ?it/s]

In [9]:
pd.concat([
    results_degree,
    unbiased_results_degree
]).groupby([
    "model_name",
    "evaluation_mode",
    "use_scale_free_distribution",
    ("Model", "edge_features")
]).agg(["mean", "std"])[["auprc", "auroc", "balanced_accuracy"]]

KeyError: ('Model', 'edge_features')

<h3>Adamic Adar</h3>
<p>Here, instead of using the degree as an edge feature, we use the <a href="https://en.wikipedia.org/wiki/Adamic%E2%80%93Adar_index" target=__blank>Adamic-Adar index</a>, which is defined as the sum of the inverse logarithmic degree centrality of the neighbours shared by the two nodes. The next two cells are with and without sampling by degree distribution (as above).</p>

In [24]:
results_adamic_unnormalized = edge_prediction_evaluation(
    holdouts_kwargs=dict(train_size=0.7,edge_types=["SLI"]),
    graphs=composite_graph,
    number_of_holdouts=10,
    models=PerceptronEdgePrediction(
        edge_features=["AdamicAdar"],
        use_scale_free_distribution=False),
    use_scale_free_distribution=False,
    subgraph_of_interest=sli_subgraph
)

d = get_dict_with_values(results_adamic_unnormalized, 'Adamic Adar', False)
results_list.append(d)

Evaluating on (SLDB | HomoSapiens):   0%|                | 0/10 [00:00<?, ?it/s]

In [25]:
results_adamic_normalized = edge_prediction_evaluation(
    holdouts_kwargs=dict(train_size=0.7,edge_types=["SLI"]),    
    graphs=composite_graph,
    number_of_holdouts=10,
    models=PerceptronEdgePrediction(
        edge_features=["AdamicAdar"],
        use_scale_free_distribution=True),
    use_scale_free_distribution=True,
    subgraph_of_interest=sli_subgraph
)
d = get_dict_with_values(results_adamic_normalized, 'Adamic Adar', True)
results_list.append(d)

Evaluating on (SLDB | HomoSapiens):   0%|                | 0/10 [00:00<?, ?it/s]

<h3>Jaccard similarity coefficient</h3>
<p>The next two cells are analogous, but use the <a href="https://en.wikipedia.org/wiki/Jaccard_index" target=__blank>Jaccard similarity coefficient</a>, defined as the ratio of Intersection over Union of neighbors of the two nodes that define an edge.</p>

In [12]:
results_jaccard = edge_prediction_evaluation(
    holdouts_kwargs=dict(train_size=0.7,edge_types=["SLI"]),  
    graphs=composite_graph,
    number_of_holdouts=10,
    models=PerceptronEdgePrediction(
        edge_features=["JaccardCoefficient"],
        use_scale_free_distribution=False),
    use_scale_free_distribution=False,
    subgraph_of_interest=sli_subgraph
)
d = get_dict_with_values(results_jaccard, 'Jaccard', False)
results_list.append(d)

Evaluating on (SLDB | HomoSapiens):   0%|                | 0/10 [00:00<?, ?it/s]

In [13]:
results_jaccard_normalized = edge_prediction_evaluation(
    holdouts_kwargs=dict(train_size=0.7,edge_types=["SLI"]),  
    graphs=composite_graph,
    number_of_holdouts=10,
    models=PerceptronEdgePrediction(
        edge_features=["JaccardCoefficient"],
        use_scale_free_distribution=True),
    use_scale_free_distribution=True,
     subgraph_of_interest=sli_subgraph
)
d = get_dict_with_values(results_jaccard_normalized, 'Jaccard', True)
results_list.append(d)

Evaluating on (SLDB | HomoSapiens):   0%|                | 0/10 [00:00<?, ?it/s]

<h2>Summary</h2>

In [20]:
df = pd.DataFrame.from_dict(results_list)
#df.set_index(['name']);
df.head(20)

,name,test.mean,test.std,train.mean,train.std,normalized
0,Adamic Adar,0.500000,0.000000,0.500000,0.000000,False
1,Adamic Adar,0.464578,0.006581,0.470222,0.002838,True
2,Jaccard,0.500000,0.000000,0.500000,0.000000,False
3,Jaccard,0.499319,0.000718,0.499416,0.000308,True
